# Lesson 2.2:
# PowerGrid Models API - Using JSON Queries

This tutorial introduces the PowerGrid Models API and how it can be used to query model data.

__Learning Objectives:__

At the end of the tutorial, the user should be able to use the PowerGrid Models API to

* Query for feeder information
* Query for equipment information
* Query for device measurements

## Getting Started

Before running any of the sample routines in this tutorial, it is first necessary to start the GridAPPS-D Platform and establish a connection to this notebook so that we can start passing calls to the API.

_Open the Ubuntu terminal and start the GridAPPS-D Platform if it is not running already:_

`cd gridappsd-docker`

~/gridappsd-docker$ `./run.sh -t develop`

_Once containers are running,_

gridappsd@[container]:/gridappsd$ `./run-gridappsd.sh`

In [12]:
# Establish connection to GridAPPS-D Platform:
from gridappsd import GridAPPSD
gapps = GridAPPSD("('localhost', 61613)", username='system', password='manager')
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries

---
# Table of Contents

* [1. Introduction to the PowerGrid Model API](#1.-Introduction-to-the-PowerGrid-Model-API)


* [2. Using the PowerGrid Model API](#2.-Using-the-PowerGrid-Model-API)
    * [2.1. Specifying the Topic](#2.1.-Specifying-the-Topic)
    * [2.2. Structure of a Query Message](#2.2.-Structure-of-a-Query-Message)
    * [2.3. Specifying the requestType](#2.3.-Specifying-the-requestType)
    

* [3. Querying for Feeder Model Info](#3.-Querying-for-Feeder-Model-Info)
    * [3.1. Query for mRIDs of all Models](#3.1.-Query-for-mRIDs-of-all-Models)
    * [3.2.-Query for Details Dictionary of all Models](#3.2.-Query-for-Details-Dictionary-of-all-Models)
    
    
* [4. Querying for Object Info](#4.-Querying-for-Object-Info)
    * [4.1. Query for CIM Classes of Objects in Model](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)
    * [4.2. Query for mRIDs of Objects in a Feeder](#4.2.-Query-for-mRIDs-of-Objects-in-a-Feeder)
    * [4.3. Query for CIM Attributes of an Object](#4.3.-Query-for-CIM-Attributes-of-an-Object)
    * [4.4. Query for Object Dictionary](#4.4.-Query-for-Object-Dictionary)
    
    
* [5. Querying for Object Measurements](#5.-Querying-for-Object-Measurements)
    * [5.1. Object mRIDs vs Measurement mRIDs](#5.1.-Object-mRIDs-vs-Measurement-mRIDs)
    * [5.2. Querying for Measurements](#5.2.-Querying-for-Measurements)
    * [5.3. Filtering Returned Data](#5.3.-Filtering-Returned-Data)
    
    
* [6. GridAPPSD-Python Shortcut Methods](#6.-GridAPPSD-Python-Shortcut-Methods)
    * [6.1. Querying for mRIDs of all Models](#6.1.-Querying-for-mRIDs-of-all-Models)
    * [6.2. Query for CIM Classes of Objects in Model](#6.2.-Query-for-CIM-Classes-of-Objects-in-Model)
    * [6.3. Query for CIM Attributes of an Object](#6.3.-Query-for-CIM-Attributes-of-an-Object)

---
# 1. Introduction to the PowerGrid Model API


The PowerGrid Model API is used to pull model information, inlcuding the names, mRIDs, measurements, and nominal values of power system equipment in the feeder (such as lines, loads, switches, transformers, and DERs).


---
# 2. Using the PowerGrid Model API

## 2.1. Specifying the Topic

All queries passed to the PowerGrid Models API need to use the correct topic, which is obtained using the GridAPPS-D Topics library. For a review of GridAPPS-D topics, see Lesson 1.4 

In [3]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

[Return to Top](#Table-of-Contents)

## 2.2. Structure of a Query Message

Queries passed to PowerGrid Models API are formatted as python dictionaries or equivalent JSON scripts wrapped as a python string. 

```
message = {
    "requestType": "INSERT QUERY HERE",
    "resultFormat": "JSON",
    "modelId": "OPTIONAL INSERT MODEL mRID HERE",
    "objectType": "OPTIONAL INSERT CIM CLASS HERE",
    "objectId": "OPTIONAL INSERT OBJECT mRID HERE",
    "filter": "OPTIONAL INSERT SPARQL FILTER HERE"
}
```

The components of the message are as follows:

* `"requestType":` -- Specifies the type of query. Available requestType are listed in the next section.


* `"resultFormat":` -- Specifies the format of the response, can be `"JSON"`, `"CSV"`, or `"XML"`. (CAUTION: the PowerGridModel API uses the key _resultFormat_, while the Timeseries API uses the key _reponseFormat_. Using the wrong key for either API will result in a java.lang error.)


* `"modelID":` -- Optional. Used to filter the query to only one particular model whose mRID is specified. Be aware of spelling and capitalization differences between JSON query spelling `"modelId"` and Python Library spelling `model_id`.


* `"objectType":` -- Optional. Used to filter the query to only one CIM class of equipment. Speciying the _objectID_ will override any values specified for _objectType_. 


* `"objectID":` -- Optional. Used to filter the query to only one object whose mRID is specified. Specifying the _objectID_ will override any values specified for _objectType_. 


* `"filter":` -- Optional. Used to filter the query using a SPARQL filter. SPARQL queries are covered in the next lesson.


The usage of each of these message components are explained in detail with code block examples below. 

__Important__: Be sure to pay attention to placement of commas ( __,__ ) at the end of each JSON line. Commas are placed at the end of each line _except_ the last line. Incorrect comma placement will result in a JsonSyntaxException. 

All of the queries are passed to the PowerGrid Model API using the `.get_response(topic, message)` method for the GridAPPS-D platform connection variable.

[Return to Top](#Table-of-Contents)

## 2.3. Specifying the `requestType`

Below are the possible `requestType` strings that are used to specify the type of each query. Executable code block examples are provided for each of the requests in the subsections below.

The first group of _requestType_ key-value pairs are for queries for information related to the entire model or a set of models, such as the model name, mRID, region, and substation:

* `"requestType": "QUERY_MODEL_NAMES"` -- [Query for the list of all model name mRIDs](#3.1.-Query-for-mRIDs-of-all-Models)


* `"requestType": "QUERY_MODEL_INFO"` -- [Query for the dictionary of all details for all feeders in Blazegraph](#3.2.-Query-for-Details-Dictionary-of-all-Models)


The second group of _requestType_ key-value pairs are for queries for a single object or a single class of objects withing a model, such as the object mRID, CIM attributes, or measurement points:

* `"requestType": "QUERY_OBJECT_TYPES"` -- [Query for the types of CIM classes of objects in the model](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)


* `"requestType": "QUERY_OBJECT_IDS"` -- [Query for a list of all mRIDs for objects of a CIM class in the model](#4.2.-Query-for-mRIDs-of-Objects-in-a-Feeder)


* `"requestType": "QUERY_OBJECT"` -- [Query for CIM attributes of an object using its unique mRID](#4.3.-Query-for-CIM-Attributes-of-an-Object)


* `"requestType": "QUERY_OBJECT_DICT"` -- [Query for the dictionary of all details for an object using either its _objectType_ OR its _objectID_](#4.4.-Query-for-Object-Dictionary)


* `"requestType": "QUERY_OBJECT_MEASUREMENTS"` -- [Query for all measurement types and mRIDs for an object using either its _objectType_ OR its _ObjectID_](#5.2.-Querying-for-Measurements)

The third group of _requestType_ key-value pairs are for queries based on SPARQL filters or complete SPARQL queries. The structure of SPARQL was introduced in Lesson 1.7 (to be completed soon). Usage of these two _requestType_ will covered separately in the next two lessons.

* `"requestType": "QUERY_MODEL"` -- Query for all  part of a specified model, filtered by object type using a SPARQL filter.


* `"requestType": "QUERY"` -- Query using a complete SPARQL query.




[[Return to Top](#Table-of-Contents)]

---
# 3. Querying for Feeder Model Info


This section outlines the pre-built JSON queries that can be passed to the PowerGrid Model API to obtain mRIDs and other information for all models and feeders stored in the Blazegraph Database.

## 3.1. Query for mRIDs of all Models

This query obtains a list of all the model MRIDs stored in the Blazegraph database. 

Query requestType:

* `"requestType": "QUERY_MODEL_NAMES"`

Allowed parameters:

* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Optional. Will return results as a list in the format selected. 

In [ ]:
message = {
    "requestType": "QUERY_MODEL_NAMES",
    "resultFormat": "JSON"
}

In [ ]:
gapps.get_response(topic, message)

[Return to Top](#Table-of-Contents)

## 3.2. Query for Details Dictionary of all Models

This query returns a list of names and MRIDs for all models, substations, subregions, and regions for all available feeders stored in the Blazegraph database.

Query requestType:

* `"requestType": "QUERY_MODEL_INFO"`

Allowed parameters:

* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

In [15]:
message = {
    "requestType": "QUERY_MODEL_INFO",
    "resultFormat": "JSON"
}

In [16]:
gapps.get_response(topic, message)

{'data': {'models': [{'modelName': 'acep_psil',
    'modelId': '_77966920-E1EC-EE8A-23EE-4EFD23B205BD',
    'stationName': 'UAF',
    'stationId': '_22B12048-23DF-007B-9291-826A16DBCB21',
    'subRegionName': 'Fairbanks',
    'subRegionId': '_2F8FC9BF-FF32-A197-D541-0A2529D04DF7',
    'regionName': 'Alaska',
    'regionId': '_96465E7A-6EC3-ECCA-BC27-31B53F05C96E'},
   {'modelName': 'ieee123',
    'modelId': '_C1C3E687-6FFD-C753-582B-632A27E28507',
    'stationName': 'IEEE123',
    'stationId': '_FE44B314-385E-C2BF-3983-3A10C6060022',
    'subRegionName': 'Medium',
    'subRegionId': '_1CD7D2EE-3C91-3248-5662-A43EFEFAC224',
    'regionName': 'IEEE',
    'regionId': '_73C512BD-7249-4F50-50DA-D93849B89C43'},
   {'modelName': 'ieee123pv',
    'modelId': '_E407CBB6-8C8D-9BC9-589C-AB83FBF0826D',
    'stationName': 'IEEE123',
    'stationId': '_FE44B314-385E-C2BF-3983-3A10C6060022',
    'subRegionName': 'Medium',
    'subRegionId': '_1CD7D2EE-3C91-3248-5662-A43EFEFAC224',
    'regionName': 'I

[Return to Top](#Table-of-Contents)

---
# 4. Querying for Object Info

This section outlines the pre-built JSON queries that can be passed to the PowerGrid Model API to obtain mRIDs and other information for a particular object or a class of objects for one or more feeders stored in the Blazegraph Database.

All of the examples in this section use the IEEE 13 node model. The python constructor %s is used for all queries to enable the code block to be cut and paste into any python script without needing to change the model mRID.

In [ ]:
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries

## 4.1. Query for CIM Classes of Objects in Model

This query is used to query for a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

Query requestType is

* `"requestType": "QUERY_OBJECT_TYPES"`

Allowed parameters are

* `"modelId":` "model name mRID" -- Optional. Searches only the particular model identified by the given unique mRID
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected. 

__1) Query entire Blazegraph database__

Omit the "modelId" parameter to search the entire blazegraph database. 

In [ ]:
message = {
    "requestType": "QUERY_OBJECT_TYPES",
    "resultFormat": "JSON"
}

In [ ]:
gapps.get_response(topic, message)

__2) Query for only a particular model__

Specify the model MRID as a python string and pass it as a parameter to the method to return only the CIM classes of objects in that particular model. 

Be aware of spelling and capitalization differences between JSON query spelling `"modelId"` and Python Library spelling `model_id`.

In [20]:
message = {
    "requestType": "QUERY_OBJECT_TYPES",
    "modelId": model_mrid,
    "resultFormat": "JSON"
}

In [21]:
gapps.get_response(topic, message)

{'data': {'objectTypes': ['http://iec.ch/TC57/CIM100#ConnectivityNode',
   'http://iec.ch/TC57/CIM100#PowerElectronicsConnection',
   'http://iec.ch/TC57/CIM100#SynchronousMachine',
   'http://iec.ch/TC57/CIM100#PowerTransformer',
   'http://iec.ch/TC57/CIM100#TransformerTank',
   'http://iec.ch/TC57/CIM100#LinearShuntCompensator',
   'http://iec.ch/TC57/CIM100#ACLineSegment',
   'http://iec.ch/TC57/CIM100#LoadBreakSwitch',
   'http://iec.ch/TC57/CIM100#EnergyConsumer',
   'http://iec.ch/TC57/CIM100#EnergySource']},
 'responseComplete': True,
 'id': '1134494105'}

[Return to Top](#Table-of-Contents)

## 4.2. Query for mRIDs of Objects in a Feeder

This query is used to obtain all the mRIDs of objects of a particular CIM class in the feeder. 

Query responseType is

* `"requestType": "QUERY_OBJECT_IDS"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models
* `"objectType":` "CIM Class" -- Optional. Specifies the type of objects you wish to return details for.
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected. 

Within a particular feeder, it is possible to query for objects of all the CIM classes obtained using `"requestType": "QUERY_OBJECT_TYPES"` (discussed above in [Section 4.1](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)). Note that the RDF URI is not included in the query, only the name of the class, such as `"objectType": "ACLineSegment"` or `"objectType": "LoadBreakSwitch"`.

In [6]:
message = {
    "requestType": "QUERY_OBJECT_IDS",
    "resultFormat": "JSON",
    "modelId": model_mrid,
    "objectType": "LoadBreakSwitch"
} 

In [7]:
response_obj = gapps.get_response(topic, message)

In [8]:
response_obj['data']

{'objectIds': ['_2858B6C2-0886-4269-884C-06FA8B887319',
  '_517413CB-6977-46FA-8911-C82332E42884']}

In [9]:
switch_mrids = response_obj['data']['objectIds']

[Return to Top](#Table-of-Contents)

## 4.3. Query for CIM Attributes of an Object

This query is used to obtain all the attributes and mRIDs of those attributes for a particular object whose mRID is specified. 

Query responseType is

* `"requestType": "QUERY_OBJECT"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models
* `"objectId":` "object mRID" -- Optional. Specifies the type of objects you wish to return details for.
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

Within a particular feeder, it is possible to query for objects of all the CIM classes obtained using `"requestType": "QUERY_OBJECT_TYPES"` (discussed above in [Section 4.1](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)). Note that the RDF URI is not included in the query, only the name of the class, such as `"objectType": "ACLineSegment"` or `"objectType": "LoadBreakSwitch"`.

In [ ]:
object_mrid = "_2858B6C2-0886-4269-884C-06FA8B887319"
message = """
{
        "requestType": "QUERY_OBJECT",
        "resultFormat": "JSON",
        "modelId": "%s",
        "objectId": "%s"
}
""" % (model_mrid, object_mrid)

In [ ]:
message = {
        "requestType": "QUERY_OBJECT",
        "resultFormat": "JSON",
        "objectId": "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"
}

In [ ]:
gapps.get_response(topic, message)

[Return to Top](#Table-of-Contents)

## 4.4. Query for Object Dictionary

This query returns a python dictionary of all the equipment attributes and mRIDs. The query can be for 1) all objects of a particular `objectType` or 2) for those connected to a particular object based on the `objectId`. 

If neither `objectType` or `objectId` is provided, the query will provide all measurements belonging to the model.
Query responseType is

* `"requestType": "QUERY_OBJECT_DICT"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models

* `"objectId":` "object mRID" -- Optional. Specifies the type of objects you wish to return details for.

* `"objectType":` "CIM Class" -- Optional. Specifies the type of objects you wish to return details for.

* `"resultFormat":` "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

Speciying the `objectID` will override any values specified for `objectType`.

__Example 1: Querying for model dictionary for an `"objectID":`__

In [ ]:
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "objectId": switch_mrids[1]
}

gapps.get_response(topic, message)

__Example 2: Querying for model dictionary for an `"objectType":`__

In [29]:
model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "objectType": "TransformerTank"
}

gapps.get_response(topic, message)

{'data': [{'id': '_7783128E-2906-46F4-8747-68F2B5B6CBCE',
   'Equipment.EquipmentContainer': '_C1C3E687-6FFD-C753-582B-632A27E28507',
   'IdentifiedObject.mRID': '_7783128E-2906-46F4-8747-68F2B5B6CBCE',
   'IdentifiedObject.name': 'reg4c',
   'PowerSystemResource.Location': '_34F4F487-2436-4E8B-99EC-070AC21E5845',
   'TransformerTank.PowerTransformer': '_B89A2F76-07E6-4BC5-86FD-FC2B400B09D8',
   'type': 'TransformerTank'},
  {'id': '_9C8F8334-5AE7-49C8-8946-4A2429D95262',
   'Equipment.EquipmentContainer': '_C1C3E687-6FFD-C753-582B-632A27E28507',
   'IdentifiedObject.mRID': '_9C8F8334-5AE7-49C8-8946-4A2429D95262',
   'IdentifiedObject.name': 'reg2a',
   'PowerSystemResource.Location': '_89A034CF-A07F-407C-A956-FA0B5C013D14',
   'TransformerTank.PowerTransformer': '_8ABD7FA9-DD4A-44A0-AC71-E423F050B0CE',
   'type': 'TransformerTank'},
  {'id': '_A0AFD5F7-EFC2-4711-95DF-82F6A19A8B3F',
   'Equipment.EquipmentContainer': '_C1C3E687-6FFD-C753-582B-632A27E28507',
   'IdentifiedObject.mRID': 

[Return to Top](#Table-of-Contents)

---
# 5. Querying for Object Measurements



## 5.1. Object mRIDs vs Measurement mRIDs

A key concept in GridAPPS-D and CIM XML power system models is the difference between the object mRID of a piece of equipment and multiple measurement mRIDs associated with its control settings and power flow values.

Measurements differ from the state variables (e.g. those obtained from State Estimator or a power flow calculation) in that the values are measured here and not calculated or estimated. Each Measurement is associated to a _PowerSystemResource_, and in GridAPPS-D for now, also a Terminal that belongs to the same _PowerSystemResource_. (Non-electrical measurements, for example weather, would not have the Terminal). 

The _measurementType_ is a string code from IEC 61850, with the following currently suppported:

* __PNV__ -- Phase to Neutral Voltage


* __VA__  -- Volt-Amperes (apparent power)


* __A__ -- Amperes (current)


* __POS__ -- Position for switches and transformer taps

Each measurement object has a __name__, __mRID__, and __phases__. In GridAPPS-D, each phase is measured individually so multi-phase codes like ABC should not be used. 

Pos measurements will be discrete, for such things as tap position, switch position, or capacitor bank position. 

The others will be Analog, with magnitude and optional angle in degrees. 

Each MeasurementValue will have a timeStamp and mRID inherited from IdentifiedObject, so the values can be traced.



[Return to Top](#Table-of-Contents)

## 5.2. Querying for Measurements

This query returns details for the measurements within a model. The query can be for 1) all objects of a particular `objectType` or 2) for those connected to a particular object based on the `objectId`. 

If neither `objectType` or `objectId` is provided, the query will provide all measurements belonging to the model.
Query responseType is

* `"requestType": "QUERY_OBJECT_MEASUREMENTS"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models

* `"objectId":` "object mRID" -- Optional. Specifies the type of objects you wish to return details for.

* `"objectType":` "CIM Class" -- Optional. Specifies the type of objects you wish to return details for.

* `"resultFormat":` "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

Speciying the `objectID` will override any values specified for `objectType`.

__Example 1: Querying for all measurements for an `"objectID":`__

In [10]:
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "resultFormat": "JSON",
    "objectId": switch_mrids[1]
}

gapps.get_response(topic, message)

{'data': [{'measid': '_9999fd85-b2ab-434e-a1eb-e5a879052264',
   'type': 'A',
   'class': 'Analog',
   'name': 'LoadBreakSwitch_671692_Current',
   'bus': '671',
   'phases': 'A',
   'eqtype': 'LoadBreakSwitch',
   'eqname': '671692',
   'eqid': '_517413CB-6977-46FA-8911-C82332E42884',
   'trmid': '_F1D6C919-22FA-4E94-81B1-36823F5A9FF5'},
  {'measid': '_cfbc0267-18cf-4223-a359-93827b91595a',
   'type': 'A',
   'class': 'Analog',
   'name': 'LoadBreakSwitch_671692_Current',
   'bus': '671',
   'phases': 'B',
   'eqtype': 'LoadBreakSwitch',
   'eqname': '671692',
   'eqid': '_517413CB-6977-46FA-8911-C82332E42884',
   'trmid': '_F1D6C919-22FA-4E94-81B1-36823F5A9FF5'},
  {'measid': '_a93ba38d-1c92-4753-9be7-701316fe400d',
   'type': 'A',
   'class': 'Analog',
   'name': 'LoadBreakSwitch_671692_Current',
   'bus': '671',
   'phases': 'C',
   'eqtype': 'LoadBreakSwitch',
   'eqname': '671692',
   'eqid': '_517413CB-6977-46FA-8911-C82332E42884',
   'trmid': '_F1D6C919-22FA-4E94-81B1-36823F5A9

In [ ]:
switch_mrids[1]

__Example 2: Querying for all measurements for an `"objectType":`__

In [4]:
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "resultFormat": "JSON",
    "objectType": "ACLineSegment"
}

gapps.get_response(topic, message)

{'data': [{'measid': '_83e4ca18-56b5-49a9-8a59-8d522279a43c',
   'type': 'PNV',
   'class': 'Analog',
   'name': 'ACLineSegment_632633_Voltage',
   'bus': '633',
   'phases': 'A',
   'eqtype': 'ACLineSegment',
   'eqname': '632633',
   'eqid': '_FBE26B35-13AB-457D-9795-DF58B28E309D',
   'trmid': '_F0E10483-D8AD-46BE-AF5F-08228955796B'},
  {'measid': '_e6f41ee7-e162-431a-8338-d1d41c272443',
   'type': 'PNV',
   'class': 'Analog',
   'name': 'ACLineSegment_632633_Voltage',
   'bus': '633',
   'phases': 'B',
   'eqtype': 'ACLineSegment',
   'eqname': '632633',
   'eqid': '_FBE26B35-13AB-457D-9795-DF58B28E309D',
   'trmid': '_F0E10483-D8AD-46BE-AF5F-08228955796B'},
  {'measid': '_c3838d80-c959-44e5-920e-7d69f621bc6a',
   'type': 'PNV',
   'class': 'Analog',
   'name': 'ACLineSegment_632633_Voltage',
   'bus': '633',
   'phases': 'C',
   'eqtype': 'ACLineSegment',
   'eqname': '632633',
   'eqid': '_FBE26B35-13AB-457D-9795-DF58B28E309D',
   'trmid': '_F0E10483-D8AD-46BE-AF5F-08228955796B'},

[Return to Top](#Table-of-Contents)

## 5.3. Filtering Returned Data

After receiving the python dictionary of measurements, it will be necessary to parse it to inlcude just the desired set of measurements. This is done using the method presented in [Lesson 1.5, Section 3](Lesson%201.5.%20GridAPPS-D%20Messages.ipynb#3.-Parsing-Returned-Data)

In [11]:
obj_msr_ACline = gapps.get_response(topic, message, timeout=10)

# Filter to just values for 'data' key
obj_msr_ACline = obj_msr_ACline['data'] 

# Chose specific measurement mrid. Screen out those whose type is not PNV. For example,
obj_msr_ACline = [k for k in obj_msr_ACline if k['type'] == 'Pos']

obj_msr_ACline

[{'measid': '_95cd5a84-9c49-48e2-b7e1-f20c62230182',
  'type': 'Pos',
  'class': 'Discrete',
  'name': 'LoadBreakSwitch_671692_State',
  'bus': '671',
  'phases': 'A',
  'eqtype': 'LoadBreakSwitch',
  'eqname': '671692',
  'eqid': '_517413CB-6977-46FA-8911-C82332E42884',
  'trmid': '_F1D6C919-22FA-4E94-81B1-36823F5A9FF5'},
 {'measid': '_3b72a23c-4e29-4c7a-b484-fc1d7293bfeb',
  'type': 'Pos',
  'class': 'Discrete',
  'name': 'LoadBreakSwitch_671692_State',
  'bus': '671',
  'phases': 'B',
  'eqtype': 'LoadBreakSwitch',
  'eqname': '671692',
  'eqid': '_517413CB-6977-46FA-8911-C82332E42884',
  'trmid': '_F1D6C919-22FA-4E94-81B1-36823F5A9FF5'},
 {'measid': '_b54fb47b-0e27-4b25-907b-e7d64df40997',
  'type': 'Pos',
  'class': 'Discrete',
  'name': 'LoadBreakSwitch_671692_State',
  'bus': '671',
  'phases': 'C',
  'eqtype': 'LoadBreakSwitch',
  'eqname': '671692',
  'eqid': '_517413CB-6977-46FA-8911-C82332E42884',
  'trmid': '_F1D6C919-22FA-4E94-81B1-36823F5A9FF5'}]

[Return to Top](#Table-of-Contents)

---
# 6. GridAPPSD-Python Shortcut Methods

A small number of simple PowerGrid Model API queries have pre-built Python functions that can be used without specifying the topic and a particular message.


## 6.1. Querying for mRIDs of all Models

The `.query_model_names` method is associated with the GridAPPSD connection object and returns a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.


This method will return identical results to the python dictionary message explained above in [Section 3.1](#3.1.-Query-for-mRIDs-of-all-Models)

In [ ]:
gapps.query_model_names()

[Return to Top](#Table-of-Contents)

## 6.2. Query for CIM Classes of Objects in Model

The `.query_object_types` method is associated with the GridAPPSD connection object and returns a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

This method will return identical results to the python dictionary message explained above in [Section 4.1](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)

Allowed parameters are

* model_id (optional) - when specified, it searches only the particular model identified by the given unique mRID

__1) Query entire Blazegraph database__

Leave the arguments blank to search all models in the Blazegraph database

In [ ]:
gapps.query_object_types()

__2) Query for only a particular model__

Specify the model MRID as a python string and pass it as a parameter to the method to return only the CIM classes of objects in that particular model

In [ ]:
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries
gapps.query_object_types(model_mrid)

[Return to Top](#Table-of-Contents)

## 6.3. Query for CIM Attributes of an Object

The `.query_object` method is associated with the GridAPPSD connection object and returns a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

This method will return identical results to the python dictionary message explained above in [Section 4.3](#4.3.-Query-for-CIM-Attributes-of-an-Object)

In [ ]:
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries
object_mrid = "_2858B6C2-0886-4269-884C-06FA8B887319"

gapps.query_object(model_mrid, object_mrid)